# iGEM Wiki Scrape Analyzer

A Jupyter notebook that analyzes information pulled down by the igem-wikiscraper tool.

Dependencies:
- pandas
- qgrid
- numpy

In [53]:
import csv
import pandas as pd
import qgrid
import re
import numpy as np

In [82]:
scrapefile = open('output/descriptions_7-3-2018.csv', 'r')
#scrapereader = csv.reader(scrapefile, delimiter='|', quotechar='"')
scrapeframe = pd.read_csv(scrapefile, delimiter='|', quotechar='"') # read in CSV file
scrapeframe = scrapeframe.fillna('') # Replace NaNs with empty strings

keywords = [
    "ai2",
    "quorum",
    "autoinduction",
    "biomanufacturing",
    "lsr",
    "w3110"
]
patterns = [
    r"ai-?2", #find anything that looks like "ai2" or "ai-2"
    r"quor(um|a) ?sens(ing|or|e)",
    r"autoinduc(er|tion|es|ed|t)",
    r"(bio)?manufactur(e|ing|er)",
    r"lsr",
    r"w3110"
]
patterns = [re.compile(x, re.IGNORECASE) for x in patterns]
kpdict = dict(zip(keywords, patterns))

# Generates a series of two new columns which can be concatenated onto the original
# dataframe
def countmatches(item1, item2):
    keywordlist = []
    matches = 0
    for key, pattern in kpdict.items():
        matchitem1 = pattern.findall(item1, re.IGNORECASE)
        matchitem2 = pattern.findall(item2, re.IGNORECASE)
        if len(matchitem1) > 0 or len(matchitem2) > 0:
            keywordlist.append(key)
        matches += len(matchitem1) + len(matchitem2)
    keywordjoin = ', '.join(keywordlist)
    return pd.Series({'Matches':matches, 'Keywords':keywordjoin})

# Apply countmatches to columns 'Homepage' and '/Description'
scrapeframe = pd.concat([scrapeframe, scrapeframe.apply(lambda row: countmatches(row['Homepage'], row['/Description']), axis=1)], axis=1)

In [83]:
qgrid_widget = qgrid.show_grid(scrapeframe, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'rowHeight': 28, 'fullWidthRows': True, 'highlightSelectedCell': False, 'minVisibleR…